In [0]:
%sql
-- Auto Loader → Bronze streaming table (SQL cloud_files).
-- Run after the Unity setup so Spark writes into the governed catalog and schema.
USE CATALOG main_nyctaxi;
-- The raw schema is the Bronze landing zone that holds immutable Delta tables for ingestion.
USE SCHEMA raw;
-- Ensure the Volume exists: main_nyctaxi.raw.nyctaxi_landing
-- Path form: /Volumes/<catalog>/<schema>/<volume>/[subdirs]
-- Spark Structured Streaming powers Auto Loader, so the streaming table stays in sync with new files.
CREATE OR REFRESH STREAMING TABLE taxi_raw
-- Delta handles ACID transactions and schema evolution for the Bronze layer.
TBLPROPERTIES (pipelines.autoOptimize.managed = true)
AS
SELECT *
FROM STREAM cloud_files(
  '/Volumes/main_nyctaxi/raw/nyctaxi_landing/', -- your governed landing folder
  'parquet',
  map(
    'cloudFiles.schemaLocation','/Volumes/main_nyctaxi/raw/nyctaxi_landing/_schemas/bronze',
    'cloudFiles.schemaEvolutionMode','addNewColumns'
     -- omit useNotifications unless you've configured file events
  )
);

